In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import lightning as L

In [10]:
from torch.utils.data import Dataset, DataLoader

In [6]:
input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

define the model:

In [2]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.l1 = nn.Linear(input_size,hidden_size)
        nn.init.kaiming_normal_(self.l1.weight)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
        nn.init.xavier_normal_(self.l2.weight)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [3]:
criterion = nn.CrossEntropyLoss()

define a lightning module:

In [4]:
class LitNeuralNet(L.LightningModule):
    def __init__(self, torch_net):
        super().__init__()
        self.torch_net = torch_net
    
    def training_step(self, batch, batch_idx):
        # training step defines the training loop
        # its independent of forward
        images,labels = batch
        images = images.view(-1,784)
        outputs = self.torch_net(images)
        loss = criterion(outputs,labels)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [7]:
# init the model
l_model = LitNeuralNet(NeuralNet(input_size, hidden_size, num_classes))

Define the dataset:

In [8]:
# MNIST 
train_dataset = torchvision.datasets.MNIST(root='../../data/mnist/',train=True,transform=transforms.ToTensor(),download=False)
test_dataset = torchvision.datasets.MNIST(root='../../data/mnist/',train=False, transform=transforms.ToTensor())

In [11]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Train the model:

In [ ]:
trainer = L.Trainer(limit_train_batches)